In [ ]:
# Define the input layers
bert_seq_pool = Input(shape=(256, 768), dtype=tf.float64, name="query_text_input")
adj_sp_tt = Input(shape=(64,), sparse=True, dtype=tf.float64, name="Adjacency_Matrix_Sparse")
statuses_count = Input(shape=(1,), dtype=tf.float32, name="statuses_count_input")
followers_count = Input(shape=(1,), dtype=tf.float32, name="followers_count_input")
friends_count = Input(shape=(1,), dtype=tf.float32, name="friends_count_input")
favourites_count = Input(shape=(1,), dtype=tf.float32, name="favourites_count_input")

enc = tf.concat([enc_bert_pool,statuses_count, followers_count, friends_count, favourites_count], axis=-1)
adj, adj_sparse = get_adj(enc)

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_adj(enco):
    # Move data to GPU
    with tf.device('/GPU:0'):  # Change '/GPU:0' to the appropriate GPU device if needed
        norm_enco = enco / np.linalg.norm(enco, axis=1, keepdims=True)
        similarity_matrix = cosine_similarity(enco)
        #adjacency_matrix = similarity_matrix
        adjacency_matrix = np.where(similarity_matrix > 0.8, 1, 0)
        adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
        adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix, adjacency_matrix_sparse

In [ ]:
rest_enc = tf.concat([statuses_count, followers_count, friends_count, favourites_count], axis=-1)

# Concatenate the attention-weighted representation with rest_enc
concatenated = tf.keras.layers.Concatenate()([rest_enc, enc_bert_pool])

# Define your deep layers with dropout rate 0.5
gso = GraphSageConv(channels=782)([concatenated, adj_sp_tt])

do = Dense(782, activation="relu", name="Dense_Layer_1")(gso)
do = BatchNormalization()(do)
do = Dropout(0.3)(do)